# 단어의 임베딩
- 빈도수 계산 : 빈도기반 -TF
- TDM : matrix TF를 행렬로 만든 것 , 사전을 이용한 단순빈도
- TF-IDF : TF*IDF
- IDF : 역문서 빈도

In [1]:
text = "John likes to watch movies. Mary likes movies too. Mary also likes to watch football games."
words = text.replace('.', '').split()
words

['John',
 'likes',
 'to',
 'watch',
 'movies',
 'Mary',
 'likes',
 'movies',
 'too',
 'Mary',
 'also',
 'likes',
 'to',
 'watch',
 'football',
 'games']

In [2]:
import numpy as np
word_count = np.unique(words, return_counts=True)
print(word_count)

(array(['John', 'Mary', 'also', 'football', 'games', 'likes', 'movies',
       'to', 'too', 'watch'], dtype='<U8'), array([1, 2, 1, 1, 1, 3, 2, 2, 1, 2]))


In [3]:
# 딕셔너리 TF 생성
word_to_cnt = {}
for word, cnt in zip(*word_count):
    word_to_cnt[word] = cnt
word_to_cnt

{np.str_('John'): np.int64(1),
 np.str_('Mary'): np.int64(2),
 np.str_('also'): np.int64(1),
 np.str_('football'): np.int64(1),
 np.str_('games'): np.int64(1),
 np.str_('likes'): np.int64(3),
 np.str_('movies'): np.int64(2),
 np.str_('to'): np.int64(2),
 np.str_('too'): np.int64(1),
 np.str_('watch'): np.int64(2)}

In [4]:
word_to_cnt['movies']

np.int64(2)

In [5]:
# !pip install scikit-learn pandas

# TDM

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = ["John likes to watch movies. Mary likes movies too." ,
          "Mary also likes to watch football games."]

vector = CountVectorizer()
tdm_array = vector.fit_transform(corpus).toarray()
tf_dic = vector.vocabulary_
print(tdm_array)
print(tf_dic)

[[0 0 0 1 2 1 2 1 1 1]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [10]:
import pandas as pd
tf_dic_sorted = dict(sorted(tf_dic.items(), key=lambda item: item[1]))
# tf_dic_sorted
df = pd.DataFrame(tdm_array, columns=tf_dic_sorted.keys())
df

,also,football,games,john,likes,mary,movies,to,too,watch
0,0,0,0,1,2,1,2,1,1,1
1,1,1,1,0,1,1,0,1,0,1


# TD-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()
tfidf_array = tfidf_vec.fit_transform(corpus).toarray()
tfidf_dic = tfidf_vec.vocabulary_
tfidf_dic_sorted = dict(sorted(tfidf_dic.items(),
                               key=lambda item: item[1]))

tfidf_dtm = pd.DataFrame(tfidf_array,
                         columns=tfidf_dic_sorted.keys())
tfidf_dtm

,also,football,games,john,likes,mary,movies,to,too,watch
0,0.000000,0.000000,0.000000,0.323699,0.460629,0.230315,0.647398,0.230315,0.323699,0.230315
1,0.446101,0.446101,0.446101,0.000000,0.317404,0.317404,0.000000,0.317404,0.000000,0.317404


In [13]:
!pip install --upgrade gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 7.4 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 9.3 MB/s  0:00:01m0:00:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 7.6 MB/s  0:00:05m0:00:0100:01m
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: scipy━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [numpy]
    Found existing installation: scipy 1.15.3━━━━━━━━━━━━━━━━━ 1/4 [numpy]
    Uninstalling scipy-1.15.3:90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [numpy]
      Successfully uninstalled scipy-1.15.3━━━━━━━━━━━━━━━━━━━ 1/4 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [gensim]2m3/4 [gensim]
